## Libraries Importing

In [1]:
import os
import shutil #recursively moves a file from src to dest and vice versa
import cv2 as cv 
import numpy as np
from tensorflow.keras import utils

In [2]:
def create_dir(in_dir, out_dir, gdir_1):
    for label in os.listdir(path= in_dir):
        if not os.path.exists(os.path.join(out_dir, gdir_1, label)):
            os.makedirs(os.path.join(out_dir, gdir_1, label))

In [3]:
def preprocess_train_images(in_dir = 'input/train', out_dir= 'processed'):
    gdir_1 = 'gabor_trin_images'
    
    create_dir(in_dir, out_dir, gdir_1)
    
    for label in os.listdir(path= in_dir):
        print('Processing started for : ',label)
        for image_name in os.listdir(os.path.join(in_dir,label)):
            if not image_name.endswith('.db'):
                in_path = os.path.join(in_dir, label, image_name)
                out_path = os.path.join(out_dir, gdir_1, label, image_name) #path of gabor1
                img = cv.imread(in_path,0) #reading image 
                gabor_train = cv.getGaborKernel((18, 18), 1.5, np.pi/4, 5.0, 1.5, 0, ktype=cv.CV_32F) #initialising the parameters of gabor filter 
                filtered_img_train = cv.filter2D(img, cv.CV_8UC3, gabor_train) # applying gabor filter
                cv.imwrite(out_path, filtered_img_train) #writing filtered image in out_oath
        print('------ Processing done ------')

In [4]:
def preprocess_test_images(in_dir = 'input/test', out_dir= 'processed'):
    gdir_1 = 'gabor_test_images'
    
    create_dir(in_dir, out_dir, gdir_1)
    
    for label in os.listdir(path= in_dir):
        print('Processing started for : ',label)
        for image_name in os.listdir(os.path.join(in_dir,label)):
            if not image_name.endswith('.db'):
                in_path = os.path.join(in_dir, label, image_name)
                out_path = os.path.join(out_dir, gdir_1, label, image_name) #path of gabor1
                img = cv.imread(in_path,0) #reading image 
                gabor_test = cv.getGaborKernel((18, 18), 1.5, np.pi/4, 5.0, 1.5, 0, ktype=cv.CV_32F) #initialising the parameters of gabor filter 
                filtered_img_test = cv.filter2D(img, cv.CV_8UC3, gabor_test) # applying gabor filter
                cv.imwrite(out_path, filtered_img_test) #writing filtered image in out_oath
        print('------ Processing done ------')

In [5]:
preprocess_train_images()

Processing started for :  Acne
------ Processing done ------
Processing started for :  benign
------ Processing done ------
Processing started for :  Dermatitis
------ Processing done ------
Processing started for :  Eczema
------ Processing done ------


In [6]:
preprocess_test_images()

Processing started for :  Acne
------ Processing done ------
Processing started for :  benign
------ Processing done ------
Processing started for :  Dermatitis
------ Processing done ------
Processing started for :  Eczema
------ Processing done ------


## Train Dataset Preprocessing

In [7]:
input_dir = os.path.join('processed', 'gabor_trin_images')
print('input_dir :', input_dir)

input_dir : processed\gabor_trin_images


In [8]:
no_samples = 0
labels = os.listdir(input_dir)
print('Class labels : ', labels)

for label in labels:
    total = len(os.listdir(os.path.join(input_dir, label)))
    print(label, total)
    no_samples +=total
print('-'*30)
print('Total no of samples: ',no_samples)
print('-'*30)

Class labels :  ['Acne', 'benign', 'Dermatitis', 'Eczema']
Acne 115
benign 300
Dermatitis 140
Eczema 257
------------------------------
Total no of samples:  812
------------------------------


In [9]:
img_rows = 256
img_cols = 256
channels = 3

In [10]:
def load_data_train(input_dir, no_samples, labels):
    
    images = np.ndarray((no_samples, img_rows, img_cols, channels), dtype = np.float32)
    targets = np.zeros((no_samples,), dtype = np.uint8)
    
    i = 0
    print('-'*30)
    print('Loading training images...')
    print('-'*30)
    
    for j, label in enumerate(labels):
        image_names = os.listdir(os.path.join(input_dir, label))
        total = len(image_names)
        print(label, total)
        for image_name in image_names:
            img = cv.imread(os.path.join(input_dir, label, image_name), 1)
            img = np.array(cv.resize(img, (img_rows, img_cols)))
            
            images[i] = np.reshape(a= img, newshape= (img_rows, img_cols, channels))
            targets[i] = j
            
            i += 1
    print('Loading done.')
    
    targets = utils.to_categorical(y= targets, num_classes= len(labels))
    
    return images, targets

In [11]:
X, Y = load_data_train(input_dir, no_samples, labels)

------------------------------
Loading training images...
------------------------------
Acne 115
benign 300


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 2351622 bytes in function 'cv::OutOfMemoryError'


In [ ]:
print(X.shape, Y.shape)

In [ ]:
np.savez(file= 'GaborDataset/train_data', x= X, y= Y)

## Test Dataset

In [ ]:
input_dir = os.path.join('processed', 'gabor_test_images')
print('input_dir :', input_dir)

In [ ]:
no_samples = 0
labels = os.listdir(input_dir)
print('Class labels : ', labels)

for label in labels:
    total = len(os.listdir(os.path.join(input_dir, label)))
    print(label, total)
    no_samples +=total
print('-'*30)
print('Total no of samples: ',no_samples)
print('-'*30)

In [ ]:
def load_data_test(input_dir, no_samples, labels):
    
    images = np.ndarray((no_samples, img_rows, img_cols, channels), dtype = np.float32)
    targets = np.zeros((no_samples,), dtype = np.uint8)
    
    i = 0
    print('-'*30)
    print('Loading testing images...')
    print('-'*30)
    
    for j, label in enumerate(labels):
        image_names = os.listdir(os.path.join(input_dir, label))
        total = len(image_names)
        print(label, total)
        for image_name in image_names:
            img = cv.imread(os.path.join(input_dir, label, image_name), 1)
            img = np.array(cv.resize(img, (img_rows, img_cols)))
            
            images[i] = np.reshape(a= img, newshape= (img_rows, img_cols, channels))
            targets[i] = j
            
            i += 1
    print('Loading done.')
    
    targets = utils.to_categorical(y= targets, num_classes= len(labels))
    
    return images, targets

In [ ]:
images, labels = load_data_test(input_dir, no_samples, labels)

In [ ]:
print(images.shape, labels.shape)

In [ ]:
np.savez(file= 'GaborDataset/test_data', x= images, y= labels)

# Done